In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Поведенческие паттерны  
  
  
### Определяют алгоритмы и способы реализации взаимодействия различных объектов и классов.

* Chain of Responsibility
* __Command__
* Iterator
* __Mediator__
* Memento
* __Observer__
* State
* Strategy
* Template Method
* Visitor

---

## Command

### Задача  
  
Вы разрабатываете текстовый редактор. Существует несколько способов скопировать, вырезать, вставить часть текста и тд, включая шорткаты Ctrl+C / Ctrl+X / Ctrl+V, стандартная верхняя функциональная панель, функциональное меню по клику мышки. Возникает вопрос: куда поместить код обработчиков кликов по этим кнопкам/шорткатам?  
  
Допустим, мы нашли решение. Но теперь мы хотим логгировать выполнение тех или иных действий в нашем интерфейсе. Более того, возникла потребность наши действия:
  * отменять
  * ставить в очередь
  * отложенный вызов действия
  
Подходит ли предложенное Вами решение теперь? Что плохо применяется в предложенную Вами парадигму?  
  
Давайте подумаем, а должна ли кнопка интерфейса иметь прямой доступ к данным (тексту)? Если Вы спроектировали таким образом, что прямого доступа нет, то Вы - молодец, а если нет, то давайте убирать зависимость между программным модулем, порождающим действие и модулем, выполняющем его (low coupling).

### Решение  
  
Каждый вызов, отличающийся от других, следует завернуть в собственный класс с единственным методом, который и будет осуществлять вызов. Такие объекты называют _командами_.

К объекту интерфейса можно будет привязать объект команды, который знает, кому и в каком виде следует отправлять запросы. Когда объект интерфейса будет готов передать запрос, он вызовет метод команды, а та — позаботится обо всём остальном.  
  
При нажатии кнопки будут делегировать работу связанным командам, а команды — перенаправлять вызовы тем или иным объектам бизнес-логики.

Так же можно поступить и с контекстным меню, и с горячими клавишами. Они будут привязаны к тем же объектам команд, что и кнопки, избавляя классы от дублирования.

![command](uml/command.png)
  
Отправитель (Invoker) ссылается на Команду (кнопка хранит ссылку на объект команды). Invoker работает со своей командой исключительно через общий Команды (т.е. не знает с какой конкретной Командой работает). 
Обычно команда не выполняет самостоятельно, а лишь передаёт вызов получателю, которым является один из объектов бизнес-логики (Receiver).
Клиент связывает объекты отправителей с созданными командами, а созданных командах опеределяет параметры и сам Receiver: 
```js
invoker.executeCommand() 
    -> invoker.command.execute() 
        -> invoker.command.receiver.operation(invoker.command.params)
            -> ... // конкретно бизнесс-логика
```

In [18]:
import abc
import sys


class Receiver(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def operation(self, *args, **kwargs):
        pass


class StdoutNotificationReceiver(Receiver):
    def operation(self, *args, **kwargs):
        print(args, kwargs, file=sys.stdout, flush=True)
        
        
class StderrNotificationReceiver(Receiver):
    def operation(self, *args, **kwargs):
        print(args, kwargs, file=sys.stderr, flush=True)


class TextBuffer:
    args = []
    kwargs = {}


class Invoker:
    def set_command(self, command):
        self._command = command

    def execute_command(self):
        self._command.execute()
        
    def handle(self):
        self.execute_command()


class StdoutInvoker(Invoker):
    pass


class StderrInvoker(Invoker):
    pass


class Command(metaclass=abc.ABCMeta):
    def __init__(self, receiver: Receiver):
        self._receiver = receiver

    @abc.abstractmethod
    def execute(self):
        pass


class ConcreteCommand(Command):
    def execute(self):
        args = TextBuffer.args
        kwargs = TextBuffer.kwargs
        self._receiver.operation(*args, **kwargs)


class StdoutCommand(ConcreteCommand):
    pass


class StderrCommand(ConcreteCommand):
    pass

In [19]:
out_receiver = StdoutNotificationReceiver()
stdout_command = StdoutCommand(out_receiver)
stdout_invoker = StdoutInvoker()
stdout_invoker.set_command(stdout_command)

err_receiver = StderrNotificationReceiver()
stderr_command = StderrCommand(err_receiver)
stderr_invoker = StderrInvoker()
stderr_invoker.set_command(stderr_command)

stdout_invoker.handle()
stderr_invoker.handle()

TextBuffer.args.extend([3, 1, 4])

stdout_invoker.handle()
stderr_invoker.handle()

TextBuffer.kwargs.update(a=2, b=42)

stdout_invoker.handle()
stderr_invoker.handle()

() {}


() {}


(3, 1, 4) {}


(3, 1, 4) {}


(3, 1, 4) {'a': 2, 'b': 42}


(3, 1, 4) {'a': 2, 'b': 42}


### Плюсы

 * убираем связь между отправителем и получателем
 * простая отмена и повтор операций
 * отложенный запуск операций
 * можно наследовать команды
 * команды можно сохранять

### Минусы

 * много классов

---

## Mediator

### Задачa

Вернемся к разработке графических пользовательских интерфейсов.  
  
Вы создаёте сайт, имеющий сложную модель поведения. Например, при нажатии некоторых кнопок, появляется окно регистрации/входа. При проведении в открытом окне входа — меняется плашка с именем пользователя. По нажатию на кнопку “выход” — она снова переводится в исходное состояние и так далее.  
Связей в одном окне достаточно много и передача запросов оказывается крайне запутанной.


Основные проблемы с которыми мы сталкиваемся:  
 * Хаос в классах и их связях
 * Такая сильная связанность может перерости на сильную связанность модулей 
 * Сложно повторно использовать класс из-за огромного числа связей
 * Сложность тестирования

### Решение  
  
Давайте организацию общение классов сделем не напрямую друг с другом, а через отдельный объект-посредник, который знает, кому нужно перенаправить тот или иной запрос. Благодаря этому, компоненты системы будут зависеть только от посредника, а не от десятков других компонентов.

![mediator](uml/mediator.png)

 * Каждый компонент хранит ссылку на объект посредника, но работает с ним только через абстрактный интерфейс посредников.  
 * В параметрах метода ```notify``` можно передавать детали события: ссылку на компонент, в котором оно произошло, и любые другие данные.  
 * Компоненты не должны общаться друг с другом напрямую. Если в компоненте происходит важное событие, он должен оповестить своего посредника, а тот сам решит — касается ли событие других компонентов, и стоит ли их оповещать. При этом компонент-отправитель не знает кто обработает его запрос, а компонент-получатель не знает кто его прислал.

### Плюсы

 * Распутывает зависимости между компонентами программы
 * Централизует управление, позволяет соблюдать SRP
 * Позволяет повторное использование
 * Упрощает тестрование
 
### Минусы

 * GOD Object

---

## Observer

### Задача

На ютубе появилось множество опций для подписки на канал: обычная подписка (новые материалы отражаются в отдельной вкладке), “колокольчик” с опциями “не оповещать” (тогда сообщение о новых видео появляется при входе на сайт) и “оповещать” (приходят пуш-уведомления).  
Необходимо реализовать данные модели взаимодействия

### Решение  

Список подписчиков составляется динамически, объекты могут как подписываться на определённые события, так и отписываться от них прямо во время выполнения программы через `EventManager`.

При такой реализации издатель не ведёт список подписчиков самостоятельно, а делегирует это отдельному объекту `EventManager`.
  
Для добавления в программу новых подписчиков не нужно менять классы издателей, пока они работают с подписчиками через общий интерфейс.

![observer](uml/observer.png)

### Плюсы

 * Издатели не зависят от конкретных классов подписчиков и наоборот
 * Подхватывать и отписывать объекты на лету

---

**Сравните** ``